<a href="https://colab.research.google.com/github/coregvy/Con4AI/blob/master/con4_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# コネクトフォー の ゲームAIを作ろう

## AIの準備

### tensorflow 1.x インストール
tensorflow 1系はColabで使えなくなったので、強制的にインストールする\
参考：https://qiita.com/katoyu_try1/items/0228870c41d9ac54e6e9

（例外的な処理なので、Colabで使えなくなったらゴメンネ）\
将来的には Stable Baselines3に対応するよう改修予定

In [ ]:
!pip uninstall -y tensorflow tensorflow-gpu tensorboard tensorflow-estimator
!pip install tensorflow-gpu==1.15.2 --quiet

### ライブラリのインストール

MPIは並列処理のライブラリです。

In [ ]:
!pip install gym==0.19.0 tensorflow==1.15 stable-baselines --quiet
# !pip install stable-baselines[mpi] --quiet

## ゲームAIの開発

[StableBaselines](https://stable-baselines.readthedocs.io/en/master/index.html) / [OpenAI Gym](https://github.com/openai/gym) を使用して機械学習AIを開発します。

### Dependencis

In [ ]:
#!python3.7
import tensorflow as tf;
import re
import random
import gym
import numpy as np

from stable_baselines import PPO2
from stable_baselines.common.policies import MlpPolicy

import warnings

# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import logging
tf.get_logger().setLevel(logging.ERROR)

+ tensorflow のインストール確認

In [ ]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

### game_util.py

コネクトフォー独自のルールやロジックなど

In [ ]:

class GameUtil:
  @staticmethod
  def stdinToState(stdin, blank='0', my='1', your='2'):
    ao = stdin.splitlines()
    meta = ao.pop(0).split(' ')
    return list(map(lambda x: list(re.sub('[^MB]', 'Y', x.replace('.', 'B').replace(meta[2], 'M')).replace('Y', your).replace('B', blank).replace('M', my)), ao))

  def listToState(ao, meta, blank='0', my='1', your='2'):
    for row in range(int(meta[1])):
      for col in range(int(meta[0])):
        if ao[row][col] == '.':
          ao[row][col] = blank
        elif ao[row][col] == meta[2]:
          ao[row][col] = my
        else:
          ao[row][col] = your
    return ao

  @staticmethod
  def resetState(row, col):
    return [['0'] * col for i in range(row)]

  @staticmethod
  def fallCoin(state, action, mark='1', blank='0'):
    """ Return new state

    Args:
        state (list[list[str]]): state list
        action (str): [description]
        mark (str, optional): [description]. Defaults to '1'.
        blank (str, optional): [description]. Defaults to '0'.

    Returns:
        list: new state
    """
    fallNg = True
    for ry in range(len(state)):
      y = len(state) - ry - 1
      if state[y][action] == blank:
        state[y][action] = mark
        fallNg = False
        break
    return state, fallNg

  @staticmethod
  def checkEnd(state, goal=4, blank='0'):
    """ Check if the game is finished

    Args:
        state (list[list[str]]): game state list
        goal (int, optional): goal count. Defaults to 4.
        blank (str, optional): blank mark. Defaults to '0'.

    Returns:
        str: Win mark or blank
    """
    # GameUtil.render(state)
    # check row
    for row in range(len(state)):
      for col in range(len(state[row]) - goal + 1):
        tmpMark = state[row][col]
        if tmpMark == blank:
          continue
        # print('---1:', row, col, tmpMark)
        for p in range(goal - 1):
          # print('---3:', row, col + p + 1, state[row][col + p + 1])
          if tmpMark != state[row][col + p + 1]:
            tmpMark = blank
            break
        
        # print('---2:', row, col, tmpMark)
        if tmpMark != blank:
          return tmpMark

    # check col
    for col in range(len(state[0])):
      for row in range(len(state) - goal + 1):
        tmpMark = state[row][col]
        if tmpMark == blank:
          continue
        # print('|||1', row, col, tmpMark)
        for p in range(goal - 1):
          # print('|||2:', row+p+1, col, tmpMark)
          if tmpMark != state[row + p + 1][col]:
            tmpMark = blank
            break

        if tmpMark != blank:
          return tmpMark

    # check /
    for row in range(goal - 1, len(state)):
      for col in range(0, len(state[row]) - goal + 1):
        tmp = state[row][col]
        # print('/', row, col, tmp)
        if tmp == blank:
          continue
        for r in range(1, goal):
          # print('//', row, col, tmp, r)
          if tmp != state[row - r][col + r]:
            tmp = blank
            break
        if tmp != blank:
          return tmp

    # check \
    for row in range(len(state) - goal + 1):
      for col in range(len(state[row]) - goal + 1):
        tmp = state[row][col]
        # print('\\', row, col, tmp)
        if tmp == blank:
          continue
        for r in range(1, goal):
          if tmp != state[row + r][col + r]:
            tmp = blank
            break
        if tmp != blank:
          return tmp

    return blank

  @staticmethod
  def render(state, my = '1', blank = '0'):
    print('-0-1-2-3-4-5-6-')
    for i in range(len(state)):
      print(' ', end='')
      for j in range(len(state[i])):
        mark = '☆'
        if state[i][j] == my:
          mark = '◆'
        elif state[i][j] == blank:
          mark = '・'
        print(mark, end='')
      print()
    print('--------------')

  @staticmethod
  def enemyPlay(state):
    # todo
    pos = random.randrange(7)
    if state[0][pos] == '0':
      return pos
    else:
      return GameUtil.enemyPlay(state)

### environment.py

StableBaselines の環境クラス

In [ ]:

class Con4(gym.Env):
  MY_MARK = '1'
  BLANK_MARK = '0'
  MAX_ROW = 6
  MAX_COL = 7

  def __init__(self):
    super(Con4, self).__init__()
    self.board = GameUtil.resetState(self.MAX_ROW, self.MAX_COL)
    self.action_space = gym.spaces.Discrete(self.MAX_COL)
    self.observation_space = gym.spaces.Box(low=0, high=2, shape=(self.MAX_ROW, self.MAX_COL))

  def reset(self):
    self.board = GameUtil.resetState(self.MAX_ROW, self.MAX_COL)
    return self.board

  def step(self, action):
    reward = 0
    done = False
    self.board, stepNg = GameUtil.fallCoin(self.board, action, self.MY_MARK, self.BLANK_MARK)
    if stepNg:
      # この列にコインをこれ以上落とせなかった
      done = True
      reward = -10000
      return self.board, reward, done, {}
    # 相手の行動を追加する
    self.board, stepNg = GameUtil.fallCoin(self.board, GameUtil.enemyPlay(self.board), '2', self.BLANK_MARK)
    win = GameUtil.checkEnd(self.board)
    if win == self.MY_MARK:
      # 自分が勝った
      done = True
      reward = 1.0
    elif win != self.BLANK_MARK:
      # 相手が勝った
      done = True
      reward = -1
    return self.board, reward, done, {}

  def render(self, mode='console', close=False):
    GameUtil.render(self.board, self.MY_MARK, self.BLANK_MARK)

  def initState(self):
    """ 盤面を初期化する

    Returns:
        list: 初期化された盤面の2次元配列
    """
    return [[self.BLANK_MARK] * self.MAX_COL for i in range(self.MAX_ROW)]

### training

指定回数反復学習し、結果をモデルファイルとして保存する

In [ ]:
#!python3.7
env = Con4()

# モデルの生成
#  verbose：ログの詳細表示(0:ログなし、1:訓練情報を表示、2:TensorFlowログを表示)
model = PPO2('MlpPolicy', env, verbose=0, tensorboard_log='./log')
# model = PPO2(MlpPolicy, env, verbose=0)
# モデルの学習
sample = 20000
model.learn(total_timesteps=sample)
# モデルの保存
model.save('con4_model_' + str(sample))

print('training end')


### 学習結果の確認

Tensorboard を使用して、学習の様子を確認します。\
パラメータや報酬ロジックを変更した際には違いを確認し、より強いAIになるよう調整しましょう

In [ ]:
%tensorboard --logdir=./log

## AIのテスト

作ったAIが想定通りに動くか試してみましょう

In [ ]:
state = GameUtil.resetState(6, 7)
i = 0

while True:
  i += 1
  action, _ = model.predict(state)
  state, done = GameUtil.fallCoin(state, action)
  if done:
    print('failed fall: ', action)
    GameUtil.render(state)
    break
  done = GameUtil.checkEnd(state)
  if done != '0':
    print('end: ', i)
    break

  GameUtil.render(state)
  if done != '0':
    print('win ai: ', i)
    break
  print('AI action:', done, action)
  action = input('input action > ')
  state, done = GameUtil.fallCoin(state, int(action), mark = '2')
  if done:
    print('failed fall: ', action)
    GameUtil.render(state)
    break
  done = GameUtil.checkEnd(state)
  if done != '0':
    print('win player: ', i)
    break

-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・◆・・
--------------
AI action: 0 4
input action > 2
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・☆・◆◆・
--------------
AI action: 0 5
input action > 3
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・◆・
 ・・☆☆◆◆・
--------------
AI action: 0 5
input action > 4
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・・・・
 ・・・・☆◆・
 ◆・☆☆◆◆・
--------------
AI action: 0 0
input action > 5
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・・・
 ・・・・・◆・
 ・・・・・☆・
 ・・・・☆◆・
 ◆・☆☆◆◆・
--------------
AI action: 0 5
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・☆・
 ・・・・・◆・
 ・・・・・☆・
 ・・・・☆◆・
 ◆◆☆☆◆◆・
--------------
AI action: 0 1
input action > 4
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・☆・
 ・・・・・◆・
 ・・・・☆☆・
 ・・・・☆◆・
 ◆◆☆☆◆◆◆
--------------
AI action: 0 6
input action > 4
-0-1-2-3-4-5-6-
 ・・・・・・・
 ・・・・・☆・
 ・・・・☆◆・
 ・・・・☆☆・
 ・・・◆☆◆・
 ◆◆☆☆◆◆◆
--------------
AI action: 0 3
input action > 4
win player:  8


## WebSocket準備

ゲーム画面と連携するため、WebSocketの準備をします

In [ ]:
!pip install websocket-client

In [ ]:
import websocket
import json
try:
    import thread
except ImportError:
    import _thread as thread
import time

class Websocket_Client():
    isLogin = False
    userNo = -1

    def __init__(self, host_addr):

        # デバックログの表示/非表示設定
        websocket.enableTrace(True)

        # WebSocketAppクラスを生成
        # 関数登録のために、ラムダ式を使用
        self.ws = websocket.WebSocketApp(host_addr,
            on_message = lambda ws, msg: self.on_message(ws, msg),
            on_error   = lambda ws, msg: self.on_error(ws, msg),
            on_close   = lambda ws: self.on_close(ws))
        self.ws.on_open = lambda ws: self.on_open(ws)

    # メッセージ受信に呼ばれる関数
    def on_message(self, ws, message):
        print("### receive : {}".format(message))
        sendData = {}
        msg = json.loads(message)
        if msg['call'] == 'reload':
            print('+++++ reload')
            if self.isLogin:
                print('+++ Logedin reload')
            elif 'player' in msg:
                print('ゲームが進行中です。想定外の場合はゲーム画面からゲーム終了ボタンを押してください。')
                self.ws.close()
                return
            elif 'user0' in msg:
                print('先攻のユーザがログイン中です。後攻で参加します。')
                sendData['user1'] = input('ユーザ名を入力してください >')
                sendData['call'] = 'login'
                self.isLogin = True
                self.userNo = '1'
            elif 'user1' in msg:
                print('後攻のユーザがログイン中です。先攻で参加します。')
                sendData['user0'] = input('ユーザ名を入力してください >')
                sendData['call'] = 'login'
                self.isLogin = True
                self.userNo = '0'
            else:
                self.userNo = input('先攻で参加する場合は0, 後攻で参加する場合は1を入力 > ')
                sendData['user'+self.userNo] = input('ユーザ名を入力してください >')
                sendData['call'] = 'login'
                self.isLogin = True
        elif msg['call'] == 'step':
            print('+++++ step')
            if self.isLogin == False:
                print('ゲームが進行中です。想定外の場合はゲーム画面からゲーム終了ボタンを押してください。')
                self.ws.close()
                return
            if self.userNo == str(msg['player']):
                print('action start')
                sendData['call'] = 'step'
                sendData['stdin'] = msg['stdin']
                sendData['player'] = self.userNo
                # todo
                sendData['stdout'] = input('next action >')
            else:
                print('skip action')
                return
        elif msg['call'] == 'login':
            print('+++++ login')
            return
        elif msg['call'] == 'end':
            print('+++++ end', msg['message'])
        else:
            print('+++++ unknown call', msg['call'])
            return

        if 'call' in sendData:
            self.sendJson(sendData)
            

    # エラー時に呼ばれる関数
    def on_error(self, ws, error):
        print(error)

    # サーバーから切断時に呼ばれる関数
    def on_close(self, ws):
        print("### closed ###")

    # サーバーから接続時に呼ばれる関数
    def on_open(self, ws):
        thread.start_new_thread(self.run, ())
        print('+++ start new thread end');
        time.sleep(1)
        self.sendJson({'call': 'reload'})

    # サーバーから接続時にスレッドで起動する関数
    def run(self, *args):
        while True:
            time.sleep(0.1)
            # input_data = input("send data:") 
            # self.ws.send(input_data)
    
        self.ws.close()
        print("thread terminating...")
    def sendJson(self, data):
        print('### send:', data)
        self.ws.send(json.dumps(data))

    # websocketクライアント起動
    def run_forever(self):
        self.ws.run_forever()



## プレイ

[ゲーム画面](https://www.tomiko.cf/red/con4/room/demo-a.html) を開いて、作ったAIと対戦してみよう！

In [ ]:
HOST_ADDR = "wss://www.tomiko.cf/red/api/con4/demo-a"
ws_client = Websocket_Client(HOST_ADDR)
ws_client.run_forever()
